# Processing _ABM_, _Hotmaps_, and _JRC_ data to estimate the existing heat-only generation capacities

The goal of this notebook is to calculate estimates for the necessary heat-only boiler capacities,
both in distributed heating as well as in district heating systems.
The processing relies on three primary data sources:

1. The ABM heating demand timeseries informs us about the estimated peak demand relative to the yearly demand, used for sizing the capacity.
2. The Hotmaps project results for the yearly heating demands per fuel, mapped into different heating technologies for the distributed heating.
3. The JRC-IDEES yearly district-heat-only production to estimate the district heating system capacity distribution.

## Julia environment setup

In [ ]:
## Julia environment setup

using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

using CSV
using DataFrames
using Dates
using Statistics
using Serialization
using XLSX

## Read peak-to-yearly demand ratios calculated in `demand_scaling.ipynb`

In [ ]:
## Read peak-to-yearly demand ratios

peak_path = "output/peak_to_yearly_demand_ratios_MW_GWh.csv"
peak_data = DataFrame(CSV.File(peak_path))
rows, cols = size(peak_data)
peak_data = stack(
    peak_data,
    3:cols;
    variable_name=:country,
    value_name=:peak_to_demand_ratio
)
describe(peak_data)

## Read extended Hotmaps data from `demand_scaling.ipynb`

In [ ]:
## Read extended Hotmaps data

hm_data = deserialize("hm_data.ser")
describe(hm_data)

### Examine heat pump COPs

The Hotmaps data has separate values for `Electricity` and `Ambient Heat` as the `Fuel` for heat pumps. Let's see how these assumptions vary between countries and end-uses.

In [ ]:
## Examine the heat pump COPs based on their electricity and ambient heat consumption

df = hm_data[hm_data.Technology .== "heat pumps", :]
df = unstack(df, :Fuel, :scenario_value)
df = df[df.Electricity .> 0, :]
df.COP .= (df.Electricity + df[!, Symbol("ambient heat")]) ./ df.Electricity
#sort!(df, :COP; rev=true)
describe(df)

Well that's a bit alarming: The mean COP in the Hotmaps data is above six, which seems pretty unrealistic. Furthermore, the maximum COP is infeasibly high, and weirdly seems to occur mostly for DHW.

Therefore, there's a risk that the modelled heat pump capacity is not enough to cover the demand in certain countries if I use the `Electricity` fuel as the basis for the assumed existing capacity.

In [ ]:
# Let's check space heating and DHW separately:

df_cop_demand = unstack(df, :demand, :COP)
describe(df_cop_demand)

Surprisingly, the space heating COPs are lower than DHW COPs, which shouldn't be the case. Not really sure what's going on here with the Hotmaps data. Ultimately, it might be necessary to estimate the existing heat pump capacity based on the total and the assumed technology parameter COP, but we'll see.

## Estimate generation capacities based on Hotmaps

We'll have to estimate the technology capacities based on the yearly demands and the estimated demand peak ratio.

In [ ]:
## Calculate estimated peak capacities.

hm_capacity_data = leftjoin( # Combine estimated peak demand ratio data with yearly demands.
    hm_data,
    peak_data;
    on=[:country, :demand],
    makeunique=true,
)
rename!(hm_capacity_data, :unit_1 => :unit_ratio)
hm_capacity_data.capacity_MW = ( # Calculate the estimated capacity in MW
    hm_capacity_data.scenario_value
    .* hm_capacity_data.peak_to_demand_ratio
)
describe(hm_capacity_data)

### Technology mapping

Map the estimated peak capacities to the desired technologies.

In [ ]:
## Figure out heating system mappings from Hotmaps to the desired techs.

heat_techs = lowercase.(unique(hm_data.Technology))
gas_boiler = filter(x -> contains(x, "gas boiler"), heat_techs)
bio_boiler = filter(x -> contains(x, "biomass"), heat_techs)
oil_boiler = filter(x -> contains(x, "oil boiler"), heat_techs)
air_heatpump = filter(x -> contains(x, "heat pump"), heat_techs) # Data doesn't distinguish between different heat pumps.
ground_heatpump = filter(x -> contains(x, "heat pump"), heat_techs) # Data doesn't distinguish between different heat pumps.
solar_heating = filter(x -> contains(x, "solar"), heat_techs)
electric_heating = filter(x -> contains(x, "electric"), heat_techs)
district_heating = filter(x -> contains(x, "district heating"), heat_techs)
coal_boiler = filter(x -> contains(x, "coal"), heat_techs)
air_conditioning = filter(x -> contains(x, "conditioning"), heat_techs)

In [ ]:
# Map hm_data technologies to the desired heating techs.

distributed_tech_mapping = Dict( # (Set of technologies, assumed share)
    "gas-boiler" => (gas_boiler, 1.0),
    "bio-boiler" => (bio_boiler, 1.0),
    "oil-boiler" => (oil_boiler, 1.0),
    "coal-boiler" => (coal_boiler, 1.0),
    "air-heatpump" => (air_heatpump, 0.7), # Assumed 70% market share for A2WHPs
    "ground-heatpump" => (ground_heatpump, 0.3), # Assumed 30% market share for G2WHPs
    "solar-heating" => (solar_heating, 1.0),
    "electric-heating" => (electric_heating, 1.0),
    "district-heating" => (district_heating, 1.0),
    "air-heatpump-cool" => (air_conditioning, 1.0),
)

In [ ]:
# Sum the capacities together

rename_cols = Dict(
    :Scenario => :scenario,
    :scenario_year => :scenario_year,
    :country => :country,
    :category => :building_category,
    :output_technology => :technology,
    :demand => :demand,
    :weighted_capacity_MW => :capacity,
    :capacity_unit => :unit,
    :demand_category => :demand_category,
)
heating_capacity_data = DataFrame()
# Loop over the technology mappings.
for (name, (techs, weight)) in distributed_tech_mapping
    # Filter relevant technologies.
    df = filter(
        r -> lowercase(r.Technology) in techs && lowercase(r.Fuel) != "ambient heat", # Heat pumps have two fuel rows, omitting "ambient heat" since it's likely not what Alvaro wants.
        hm_capacity_data
    )
    if isempty(df) # Skip the rest of the loop if df is empty
        continue
    end
    # Calculate the weighted capacities
    df.output_technology .= name
    df.weighted_capacity_MW .= df.capacity_MW .* weight
    df.capacity_unit .= "MW"
    df.demand_category .= (name == "district-heating" ? "district heating" : "distributed heating")
    # Final formatting
    df = df[!, collect(keys(rename_cols))] # Drop unused columns
    rename!(df, rename_cols)
    country_cols = Symbol.(unique(df.country))
    df = stack( # Avoid nonresidential private vs nonresidential public duplicate row hassle by unstack-stack summing.
        unstack(
            df,
            :country,
            :capacity;
            combine=sum
        ),
        country_cols;
        variable_name=:country,
        value_name=:capacity
    )
    append!(heating_capacity_data, df)
end
describe(heating_capacity_data)

## Read and process JRC-IDEES data

This data is used to estimate the heat-only boiler capacity distribution in district heating systems.

In [ ]:
## Read JRC district heating heat-only data for EU27

eu_selectyear = Symbol(2021) # Select latest year from the JRC 2021

# Read excels from the directory and compile them into a single dataframe
eu27 = readdir("input-data\\JRC-IDEES-2021")
filter!(s -> !contains(s, '.') && !contains(s, "EU27"), eu27)
jrc_data = DataFrame()
for country in eu27
    df = DataFrame(XLSX.readtable(
        "input-data\\JRC-IDEES-2021\\$(country)\\JRC-IDEES-2021_PowerGen_$(country).xlsx",
        "DistHeat";
        first_row = 3, # Total gross distributed heat production (GWh) - Conventional thermal
        header = false,
        column_labels = vcat(["fuel"], string.(collect(2000:2021)))
    ))
    df.country .= country
    append!(jrc_data, df)
end

# Omit totals rows
fuels_to_omit = lowercase.([
    "Conventional thermal",
    "Residual fuel oil and other petroleum products",
    "Natural gas and biogas",
    "Solid biomass and renewable waste",
    "Non-renewable wastes"
])
inds = [!in(fuel, fuels_to_omit) for fuel in lowercase.(jrc_data.fuel)]
jrc_data = jrc_data[inds, [:country, :fuel, eu_selectyear]] # We're only interested in the latest data.
jrc_data[!, eu_selectyear] = float.(jrc_data[!, eu_selectyear])
rename!(jrc_data, eu_selectyear => :total_gross_distributed_heat_production_GWh)


## Read JRC district heating heat-only data for UK

uk_selectyear = Symbol(2015) # Select latest year from the JRC 2015

# Read excels from the directory and compile them into a single dataframe
df = DataFrame(XLSX.readtable(
    "input-data\\JRC-IDEES-2015_All_xlsx_UK\\JRC-IDEES-2015_PowerGen_UK.xlsx",
    "DistHeat";
    first_row = 3, # Total gross distributed heat production (GWh) - Conventional thermal
    header = false,
    column_labels = vcat(["fuel"], string.(collect(2000:2015)))
))
df.country .= "UK"

# Omit totals rows
fuels_to_omit = lowercase.([
    "Conventional Thermal",
    "Residual Fuel Oil and Other Petroleum Products",
    "Natural gas and Biogas",
    "Wood & Waste",
    "Non-renewable wastes",
])
inds = [!in(fuel, fuels_to_omit) for fuel in lowercase.(df.fuel)]
uk = df[inds, [:country, :fuel, uk_selectyear]] # We're only interested in the latest data.
uk[!, uk_selectyear] = float.(uk[!, uk_selectyear])
rename!(uk, uk_selectyear => :total_gross_distributed_heat_production_GWh)
append!(jrc_data, uk)


## Add missing data for Norway and Switzerland based on Sweden and Austria respectively.

extrapolation_mappings = Dict( # Map Norway and Switzerland to existing data with population-based coefficients
    "NO" => ("SE", 0.52),
    "CH" => ("AT", 0.97)
)
for (c1, (c2, coeff)) in extrapolation_mappings
    df = filter(r -> r.country == c2, jrc_data)
    df.country .= c1
    df.total_gross_distributed_heat_production_GWh .*= coeff
    append!(jrc_data, df)
end
describe(jrc_data)

In [ ]:
## Check countries with no heat-only boilers according to JRC-IDEES

df = unstack(jrc_data, :country, :total_gross_distributed_heat_production_GWh)
countries = Symbol.(names(df)[2:end])
jrc_no_dh = [
    country
    for country in countries
    if sum(skipmissing(df[!,country])) ≈ 0
]

In [ ]:
## Check countries with no district heating according to Hotmaps

inds = (heating_capacity_data.demand_category .== "district heating") .* (heating_capacity_data.capacity .<= 0)
hm_no_dh = Symbol.(unique(heating_capacity_data[inds, :].country))

In [ ]:
## Countries with no capacity but demand

setdiff(jrc_no_dh, hm_no_dh)

So apparently there are a few countries that have district heating demand according to Hotmaps, but no heat-only generation capacity.
I suppose this is technically possible if all heat is generated using CHP, but unlikely?

I'll just leave these countries without existing capacity then, since the data is even less reliable than the rest. Fortunately, these countries likely don't have that much district heating demand anyhow.

## Estimate district heating heat-only capacity shares

In [ ]:
## Map JRC fuels to the desired heating technologies

dh_techs = unique(lowercase.(jrc_data.fuel))
bio_boiler_dh = filter(x -> contains(x, "wood") || contains(x, "bio"), dh_techs)
gas_boiler_dh = filter(x -> contains(x, "gas") && !contains(x, "bio"), dh_techs)
heatpump_dh = filter(x -> contains(x, "pump"), dh_techs)
elec_boiler_dh = filter(x -> contains(x, "elec"), dh_techs)
solar_dh = filter(x -> contains(x, "solar"), dh_techs)
geothermal_dh = filter(x -> contains(x, "geo"), dh_techs)
waste_dh = filter(x -> !contains(x, "wood") && contains(x, "waste") || contains(x, "msw"), dh_techs)
oil_boiler_dh = filter(x -> contains(x, " oil") && !contains(x, "shale"), dh_techs)
coal_boiler_dh = filter(x -> contains(x, "coal") || contains(x, "lignite"), dh_techs)

# Mapping JRC fuels to desired technologies
district_tech_mapping = Dict(
    "bio-boiler-DH" => bio_boiler_dh,
    "gas-boiler-DH" => gas_boiler_dh,
    "oil-boiler-DH" => oil_boiler_dh,
    "coal-boiler-DH" => coal_boiler_dh,
    "heatpump-DH" => heatpump_dh,
    "elec-boiler-DH" => elec_boiler_dh,
    "solar-DH" => solar_dh,
    "geothermal-DH" => geothermal_dh,
    "waste-DH" => waste_dh
)

In [ ]:
## Calculate totals per technology

cols = [:country, :technology, :total_gross_distributed_heat_production_GWh]
mapped_dh = DataFrame()
for (name, techs) in district_tech_mapping
    # Filter relevant technologies
    df = filter(
        r -> lowercase(r.fuel) in techs,
        jrc_data
    )
    # Skip the rest of the loop if df is empty
    isempty(df) && continue
    # Calculate totals per country
    df.technology .= name
    df = df[!, cols]
    df = unstack(
        df,
        :country,
        :total_gross_distributed_heat_production_GWh;
        combine=sum
    )
    append!(mapped_dh, df)
end


## Calculate and convert into shares per country.

countries = unique(jrc_data.country)
for country in countries
    total = sum(mapped_dh[!, country]) # Calculate total per country
    total ≈ 0 && continue # Skip scaling if total is zero
    mapped_dh[!, country] ./= total # Scale into shares per country.
    sum(mapped_dh[!, country]) ≈ 1 || @error "Scaling not working!"
end
# Stack back to format matching heating_capacity_data
mapped_dh = stack(
    mapped_dh;
    variable_name=:country,
    value_name=:share
)
describe(mapped_dh)

In [ ]:
## Let's check technology share ranges just in case

describe(unstack(mapped_dh, :technology, :share))

Seems reasonable?
Gas is the most significant DH heat-only technology both in terms of mean as well as maximum capacity shares.
Meanwhile, solar DH is understandably the smallest share.
Interestingly, oil and elec boilers have relatively low maximum shares as well,
while geothermal climbs to a maximum share of almost 40% for Belgium?!?

## Combining data and final formatting

In [ ]:
## Form the final capacity data by appending DH technology distribution

# Separate distributed heating capacities from district heating ones.
distributed_inds = heating_capacity_data.demand_category .== "distributed heating"
district_inds = heating_capacity_data.demand_category .== "district heating"
final_capacity_data = heating_capacity_data[distributed_inds, :]
district_heating_data = heating_capacity_data[district_inds, :]
rename!(district_heating_data, :technology => :old_tech)

# Combine with DH technology shares
district_heating_data = outerjoin(district_heating_data, mapped_dh; on=:country)
# Update capacity using share
district_heating_data.capacity .*= district_heating_data.share
# Append district heating capacities to distributed ones
cols = Symbol.(names(final_capacity_data))
append!(final_capacity_data, district_heating_data[!, cols])
describe(final_capacity_data)

## Project existing capacities with decommissioning.

For the European case study, we'll want to estimate the existing fleet at a select year and project it into the future with assumed decommissioning.

In [ ]:
## Project existing capacities with decommissioning.

# Config
desired_years = [2025, 2030, 2035, 2040, 2045, 2050]

# Read heating technology params to get estimated equipment decommissioning rates.
lifetimes = DataFrame(CSV.File("output/heating_technology_params.csv"))
lifetimes.lifetime_y = 1 ./ lifetimes.lifetime_y
lifetimes = rename!(
    lifetimes[:, [:technology, :year, :lifetime_y]],
    :lifetime_y => :decommission_rate
)

# Extrapolate select years
extrapolation_years = [2025, 2035, 2045]
lifetimes = unstack(lifetimes, :year, :decommission_rate)
for ey in extrapolation_years
    lifetimes[!, Symbol(ey)] = 0.5 .* (lifetimes[!, Symbol(ey-5)] +  lifetimes[!, Symbol(ey+5)])
end
lifetimes = sort(stack(
    lifetimes, names(lifetimes)[2:end]; variable_name=:scenario_year, value_name=:decommission_rate
))
lifetimes.scenario_year .= parse.(Int, lifetimes.scenario_year)

# Combine decommissioning with capacity data and simulate remaining capacities
capacity_df = sort( # Inner join to avoid including scenario years with no lifetime params.
    innerjoin(final_capacity_data, lifetimes; on=[:technology, :scenario_year])
)
capacity_df.decommission_factor .= (1 .- capacity_df.decommission_rate).^5
capacity_df.flat_decommission .= capacity_df.capacity .* (1 .- capacity_df.decommission_factor)
remaining_capacity = capacity_df[capacity_df.scenario_year .== first(desired_years), :]
flat_decommission = deepcopy(remaining_capacity.flat_decommission)
decommission_factor = deepcopy(remaining_capacity.decommission_factor)
for year in desired_years[1:end-1]
    df = remaining_capacity[remaining_capacity.scenario_year .== year, :]
    df.capacity .= max.(df.capacity .- flat_decommission, 0.0) # Flat decommissioning rates
    #df.capacity .*= decommission_factor # For relative decommissioning
    df.scenario_year .+= 5
    append!(remaining_capacity, df) 
end
describe(remaining_capacity)

## Export outputs

In [ ]:
## Export assumed existing capacity data

# Config
dgts = 2 # Number of digits when rounding exports.
export_cols = [
    :scenario,
    :scenario_year,
    :country,
    :building_category,
    :demand_category,
    :demand,
    :technology,
    :unit,
    :capacity
]

# Export table
capacity_export = final_capacity_data[:, export_cols]
capacity_export.capacity = round.(capacity_export.capacity; digits=dgts)
capacity_export = sort!(unstack(
    capacity_export,
    :country,
    :capacity,
    combine=sum # Avoid nonresidential private vs nonresidential public duplicate row hassle.
))
CSV.write("output/scenario_estimated_existing_capacities_MW.csv", capacity_export)

In [ ]:
## Export simplified existing capacity data

# Config
select_scenario = "current"
dgts = 2 # Number of digits when rounding exports.
export_cols = [
    :country,
    :technology,
    :unit,
    Symbol(2025),
    Symbol(2030),
    Symbol(2040),
    Symbol(2050)
]

# Formatting and aggregating
inds = (remaining_capacity.scenario .== select_scenario) .* (remaining_capacity.technology .!= "air-conditioning") # Omit "air-conditioning" since it has no lifetime params
simplified_export = remaining_capacity[inds, :]
cols = [:country, :technology, :unit, :scenario_year, :capacity]
simplified_export = unstack(
    simplified_export[:, cols],
    :scenario_year,
    :capacity;
    combine=(x -> round(sum(x); digits=dgts)) # Building categories and demands summed over
)

# Export table
simplified_export = sort(simplified_export[:, export_cols])
CSV.write("output/simplified_capacities_with_decommissioning_MW.csv", simplified_export)

## Tests for the output

In [ ]:
## Test that all mapped technologies are represented
# (except district-heating since it gets purposefully replaced)

final_techs = unique(capacity_export.technology)
distributed_techs = collect(keys(distributed_tech_mapping))
district_techs = collect(keys(district_tech_mapping))
mapped_techs = vcat(distributed_techs, district_techs)
only(setdiff(mapped_techs, final_techs))  == "district-heating"

In [ ]:
## Is there any meaningful difference in the data across the scenarios for the historical data?

country_cols = names(capacity_export)[8:end]
scenario_years = unique(capacity_export.scenario_year)
df = unstack(
    stack(capacity_export, country_cols; variable_name=:country, value_name=:capacity),
    :scenario,
    :capacity
)
df.sse .= (df.ambitious .- df.current).^2
diag = []
for year in scenario_years
    df_year = df[df.scenario_year .== year, :]
    share = count(df_year.sse .> 0) / length(df_year.sse)
    tot = sum(df_year.sse)
    append!(diag, year => (share, tot))
end
diag

# 2012 data practically identical across the scenarios,
# but 2015 already almost as different as the rest of the scenario years?

In [ ]:
## Check that simplified export capacity decreases

year_cols = Symbol.([2025, 2030, 2040, 2050])
year_durs = [5, 10, 10]
data = Matrix(simplified_export[:, year_cols])
decom_rates = diff(data; dims=2)
for row in eachrow(decom_rates)
    row ./= year_durs
end
rounded_decom_rates = round.(decom_rates, RoundUp; digits=1)
inds = [col .< decom_rates[:,1] for col in eachcol(rounded_decom_rates)]
count.(inds)

# Mostly works, the rounding does throw things off a bit.


In [ ]:
## Check that `heat-pump-cool` gets mapped

"air-heatpump-cool" in simplified_export.technology